#Bienvenidos al Demo del Pipeline de Cálculo Automático de Balances de Energía EONTEC

##Introduccion

Los balances de Energía se calculan con un programa de despachos y la información real de despacho de los generadores, la primera se obtiene de XM el ente regulador del mercado de energía de Colombia y el segundo, de los reportes periódicos que saca el generador, generalmente son diarios.

Tambien se averigua la diferencia del plan vs lo real, y se puede obtener el valor de la energía generada real, vs la remanente que hace falta, y que hay que cubrir con contratos de energía. 

*"Si me sobra vendo la Energía, si me falta, la tengo que comprar"*

En este proceso la **ETL** se encargará de extraer la información de las fuentes mencionadas arriba, de forma remota, posteriormente, la transformará a un formato manejable, la cruzará y filtrará, y finalmente la depositará o la disponibilizará para descarga en un destino para que los analistas o el regulador la usen.

###Fuentes

- Descarga Archivo de despacho horario de las centrales de energia consolidado
- Consumo de API de despacho programado para la fecha del ente regulador.
- Consumo de API de Precio de la energia por kWh para el día asignado

###Transformaciones

- Cambio de formato de Excel a Spark Columnar
- Cambio de formato de JSON a Spark Columnar de datos de las API.
- Combinación de datos utilizando SQL.
- Agregación, cálculos de los balances por unidad de generación (por planta)
- Identificación de operación: Compra o Venta.

###Transferencias o Cargas
- Carga por web a Bucket o Carpeta en la nube del regulador


---
##Ejercicio

Lee el código fuente e interprétalo, realiza preguntas a los ingenieros, a los profesores o al arquitecto, en caso que no lo entiendas.

---

### Preparación de prerequisitos

En esta sección se instalan los paquetes, librerías y componentes que le permitirán al pipeline realizar sus procesos internos.

In [0]:
%sh
pip install openpyxl xlrd pydrive

## Declaracion de Librerías y Variables

En estas dos celdas importamos las librerías de nuestro lenguaje (Python) y algunos frameworks como Pyspark y Pandas y definimos las variables:

- Fecha de inicio del balance
- Fecha fin del balance
- Id del archivo publico de despacho de generacion


In [0]:
import requests
import json
from pyspark.sql.functions import flatten,col,explode
import pandas as pd
import os

---
##Ejercicio

En la siguiente celda completa los siguientes datos:

1. Acorde a las fechas de tu balance y de tu archivo de generación, completa las fechas correspondientes **fecha_inicio** y **fecha_fin**
2. Sube el archivo **DespachoGeneradoraX.xlsx** a una carpeta en Google Drive, en una carpeta pública, y obten el **id del archivo** (Averigua como con GPT, Copilot, o google)
3. Abre una cuenta en **fastupload.io** y genera las llaves api para poder subir archivos, copia las api key en las variables abajo.

---

In [0]:
fecha_inicio="2024-06-09"
fecha_fin="2024-06-20"
id_archivo_balance_drive=""
api_key_1=""
api_key_2=""


# (E) Fuentes

### (*E*) Extraccion del API de Precios de la Energía

Aquí descargamos del API los datos en formato JSON, usamos una librería nativa de Python llamada Requests y con ella consumimos el api de la URL, y almacenamos el resultado en una estructura de datos denominada un dataframe, que servirá para procesamiento posterior.

In [0]:
url = f"https://www.simem.co/backend-files/api/PublicData?startDate={fecha_inicio}&endDate={fecha_fin}&datasetId=96D56E"

response = requests.get(url)
dfPreciosBolsa=spark.read.json(sc.parallelize([response.text]))
        

---
##Ejercicio

En la siguiente celda, realiza los pasos a continuación:

1. Ve a la página del SIMEM.
2. Navega y busca la página del catálogo de datos https://www.simem.co/pages/catalogodatos/51FC0A59-3A00-462C-B449-9CB8D5E007FB 
3. En la sección de datos, busca el dataset **"Despacho programado recursos de generación"**.
4. Obten el **datasetId** y actualiza la siguiente celda de código.
5. Completa el código de la celda, para que el resultado se guarde en el dataframe **dfDespachosUnidades**.
6. Imprime el resultado del dataframe para ver que hayan datos.

---


### (*E*) Extraccion del Plan de Despacho de Energía por Unidad y Central

Aqui descargamos del API los datos del despacho en formato JSON, usamos una librería nativa de Python llamada Requests y con ella consumimos el api de la URL, y almacenamos el resultado en una estructura de datos denominada un dataframe, que servirá para procesamiento posterior.

In [0]:
datasetId=""
url = f"https://www.simem.co/backend-files/api/PublicData?startdate={fecha_inicio}&enddate={fecha_fin}&datasetId={datasetId}}"

    

### (*E*) Descarga del archivo con nuestros datos reales de generación

En estas 2 celdas, descargamos de Google Drive nuestro archivo de consolidado de generación, igual que antes, usamos Requests y con ella descargamos el archivo de la URL, en este caso tenemos que traer el archivo a una ubicación local, y posteriormente lo cargamos con Pandas, un framework de procesamiento de datos de python para analítica, y lo cargamos como un dataframe de Pandas.

---
##Ejercicio

Realiza los pasos a continuación:

1. Abre tu google drive.
2. Sube el archivo de excel que estaba en el repositorio a una carpeta en tu drive. 
3. Temporalmente pon el archivo como **público** para poderlo descargar.
4. Obten el ID del archivo y reemplazalo en la sección de variables **id_archivo_balance_drive**
5. Ejecuta esa celda para actualizar las variables.
6. Ejecuta la siguiente celda y verifica que se pueda leer el archivo con pandas.

---

In [0]:

download_url = f'https://drive.google.com/uc?export=download&id={id_archivo_balance_drive}'
local_file_path = '/tmp/balances3.xlsx'
response = requests.get(download_url)
with open(local_file_path, 'wb') as file:
    file.write(response.content)

In [0]:

dfArchivoCapacidad = pd.read_excel(local_file_path,engine='openpyxl')
display(dfArchivoCapacidad)

# (T) Transformaciones


## Filtrado de Columnas

Nuestras fuentes, vienen en un estado crudo, o "raw", y hay datos, que no nos servirán para nuestro propósito, por lo cual, la primera transformación será quitar, o filtrar, los atributos que no se necesitan.

### (*T*) Filtrado de los atributos que traen el precio de bolsa


La documentación del API nos muestra que trae muchas columnas de información, denominados metadatos, necesitamos puntualmente los datos válidos para nuestro análisis. 

```
CodigoDuracion:string
CodigoPlanta:string
FechaHora:string
Valor:long
```

En estas 3 celdas, transformamos el formato utilizando funciones como aplanar o explotar, colocando alias y realizando un filtro o un select.


In [0]:
dfPreciosBolsaFiltrado=dfPreciosBolsa.select(explode(col("result.records")).alias("results")).select(col("results.*"))

Esta celda sirve para mostrar los resultados del dataframe del Precio de Bolsa Filtrado

In [0]:
display(dfPreciosBolsaFiltrado)

Esta celda registra el dataframe como una tabla o vista temporal **PrecioBolsa**, a la que podremos hacerle consultas SQL.

---
##Ejercicio
En la siguiente celda, completa los siguientes pasos:

1. Busca en internet como se crea una tabla temporal, a partir del dataframe **dfPreciosBolsaFiltrado** en Spark y crea una tabla denominada **PreciosBolsa**.

---

In [0]:
##Tu Código Aquí:
##dfPreciosBolsaFiltrado.<Usa el Dataframe como base>
##---

### (*T*) Filtrado de los atributos que traen el despacho de las plantas

Al igual que los precios de bolsa, el API de los despachos trae datos excesivos, que no necesitamos, vamos a extraer los datos de utilidad:


```
CodigoDuracion:string
CodigoPlanta:string
FechaHora:string
Valor:long
```
Al igual que en el caso anterior, en las próximas 3 celdas aplanamos estructuras, renombramos con un alias y filtramos con una selección.

In [0]:
dfResultsDespachosUnidades=dfDespachosUnidades.select(explode(col("result.records")).alias("results")).select(col("results.*"))

Celda para mostrar los resultados del Despacho de las Unidades de Generación

In [0]:
display(dfResultsDespachosUnidades)

Celda donde registramos el dataframe con los datos descargados de los despachos en la tabla **Despachos**.

---
##Ejercicio
En la siguiente celda, completa los siguientes pasos:

1. Busca en internet como se crea una tabla temporal a partir del dataframe **dfResultsDespachosUnidades** en Spark y crea una tabla denominada **despachos**.

---

In [0]:
##Tu código Aquí:
#dfResultsDespachosUnidades.<Usa el dataframe como base>
##----

### (*T*) Filtrado de Plantas

El despacho trae todas las plantas, de nuevo, tenemos que filtrar, pero ahora solo debemos traer las que nos interesan, por ejemplo Zipaquirá, Guavio, Quimbo y Chivor, de la Empresa de Energía ACME. Lo haremos con SQL.

Luego guardaremos ese filtro en otro data frame, para delimitar el set de datos para calcular el balance con las plantas que nos importan.

Primero diseñamos el SQL para filtrar con un condicional, las plantas que nos interesan:

---
##Ejercicio

Completa en las siguientes dos celdas, estos pasos:

1. Completa la query en la siguiente celda, para que filtre de los despachos, los datos de las plantas que necesitas que están en el archivo de excel.
2. Ejecuta la query y revisa los resultados, las plantas coinciden con las del archivo?.
3. Cuando tu query funcione, en la celda siguiente, averigua como ejecutar en **spark** la query y guardar el resultado en un **dataframe** con el nombre **dfDespachosAcme**

---

In [0]:
%sql
--Tu código SQL Aquí del punto 1:
--SELECT * FROM Despachos WHERE <Condición del Filtro> order by Valor,FechaHora DESC

La query resultante, la agregamos a **Spark**, para poderla trabajar con los otros datos en un dataframe llamado **dfDespachosAcme**.

In [0]:
##Agrega el código del punto 3 aquí:
#dfDespachosAcme=

---

### *Desafío*

Logra generar **dfDespachosAcme** usando Pyspark o Pandas sin SQL!

---

Con esta celda desplegamos los resultados de los despachos filtrados por planta.

In [0]:
display(dfDespachosAcme)

Registramos los despachos filtrados por planta, en la tabla **dfCapacidadDespachosAcme**

---
##Ejercicio
En la siguiente celda, completa los siguientes pasos:

1. A partir del dataframe **dfDespachosAcme** en Spark y crea una tabla denominada **dfCapacidadDespachosAcme**.

---

In [0]:
##
#dfDespachosAcme.<Agrega tu código para crear la tabla aquí>

### (*T*) Cambio de Formato Interno

Ahora, **cambiaremos** el archivo de excel con nuestra capacidad, que fue cargado como una estructura de **Pandas, a una estructura de Spark**, para poder usar SQL con los datos descargados desde XM.

Para transformarlos, **aplicaremos un esquema, que aplicará unos tipos de dato, y les dará nombres a las columnas o atributos**.

---
##Ejercicio

1. A partir del dataframe **dfTransformadoCapacidad** en Spark y crea una tabla denominada **CapacidadArchivo**, en la siguiente celda.

---

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schemaArchivo = StructType([
    StructField("useless", StringType(), True),
    StructField("fecha", StringType(), True),
    StructField("planta", StringType(), True),
    StructField("generador", StringType(), True),
    StructField("capacidad", StringType(), True),
    StructField("codigo", StringType(), True)
])
dfTransformadoCapacidad = spark.createDataFrame(dfArchivoCapacidad,schema=schemaArchivo)
#dfTransformadoCapacidad.<Agrega tu código Aquí>

### (*T*) Vamos a extraer las fechas del archivo de capacidades y quitar con un filtro las celdas que no nos sirven

El archivo de Excel **trae muchos datos innecesarios**, debemos **quitar esas celdas**, y extraer las fechas para poder utilizar esos datos del archivo con los datos del ente regulador (xm), eso lo haremos con SQL con algunas técnicas:

- Expresiones regulares.
- Sustituciones de texto.
- Condicionales.

Y posteriormente lo volvemos a dejar en un dataframe listo para utilizar.

Primero, filtramos con SQL, las celdas que no son vacías o nulas, donde el generador no esté vacío, y omitimos otros caracteres, de paso extraemos através de expresiones regulares, el día, el mes, el año y la hora.

---
##Ejercicio

1. Lee la siguiente Query, crees que se podría realizar de otra forma?, analízala, crea una copia de la celda, modifícala, compara los resultados.
2. Usando **Spark** basado en el dataframe **dfArchivoCapacidadLimpiado** crea la tabla temporal **ArchivoCapacidadLimpiado**

---

In [0]:
%sql
SELECT  b.anio,b.mes,b.dia,b.hora, b.codigo, b.capacidad  FROM 
(SELECT SUBSTR(A.FECHA, instr(A.fecha,"YEAR")+5,4) AS anio, 
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"MONTH")+6,2),',','') AS mes,
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"DAY_OF_MONTH")+13,2),',','') AS dia,
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"HOUR_OF_DAY")+12,2),',','') AS hora,
* FROM CapacidadArchivo a WHERE generador IS NOT NULL AND generador != "NaN" AND generador != "GENERADOR") b

Una vez validada nuestra query, usamos Spark para crear el dataframe **dfArchivoCapacidadLimpiado**

In [0]:
dfArchivoCapacidadLimpiado=spark.sql("""SELECT  b.anio,b.mes,b.dia,b.hora, b.codigo, b.capacidad  FROM 
(SELECT SUBSTR(A.FECHA, instr(A.fecha,"YEAR")+5,4) AS anio, 
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"MONTH")+6,2),',','') AS mes,
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"DAY_OF_MONTH")+13,2),',','') AS dia,
regexp_replace(SUBSTR(A.FECHA, instr(A.fecha,"HOUR_OF_DAY")+12,2),',','') AS hora,
* FROM CapacidadArchivo a WHERE generador IS NOT NULL AND generador != "NaN" AND generador != "GENERADOR") """)

Con el Dataframe, Creamos la tabla temporal **ArchivoCapacidadLimpiado**

In [0]:
#Tu código Aquí:
#dfArchivoCapacidadLimpiado.<Tu código para crear la tabla temporal aquí>

Paso siguiente, de la tabla con los despachos programados del ente regulador **dfCapacidadDespachosAcme**, vamos a extraer las columnas o atributos con los que vamos a cruzar la información: día, mes, año, hora y valor, junto al código de la planta.

In [0]:
%sql
SELECT  day(FechaHora) as dia_despacho, month(FechaHora) as mes_despacho, year(FechaHora) as anio_despacho,hour(FechaHora) as hora_despacho, Valor as capacidad, CodigoPlanta as codigo FROM dfCapacidadDespachosAcme

Guardamos el SQL como un dataframe **dfDespachosAcmeTransformado**

In [0]:
dfDespachosAcmeTransformado=spark.sql("""SELECT  day(FechaHora) as dia_despacho, month(FechaHora) as mes_despacho, year(FechaHora) as anio_despacho,hour(FechaHora) as hora_despacho, Valor as capacidad, CodigoPlanta as codigo FROM dfCapacidadDespachosAcme""")

Y lo registramos como una tabla **DespachosAcmeTransformado**

---
##Ejercicio
1. Agrega como es usual, el código para crear la tabla **DespachosAcmeTransformado** a partir del dataframe **dfDespachosAcmeTransformado**
---

In [0]:
##Tu código Aquí para crear la tabla temporal
#dfDespachosAcmeTransformado.<Completa aquí>

### (*T*) Cálculo del balance de energía.

Ya tenemos dos dataframes, uno con los datos del despacho que vienen del ente regulador **DespachosAcmeTransformado** y otro con los datos limpios de nuestro archivo de excel **ArchivoCapacidadLimpiado**, en esta sección, usaremos SQL para restar el despacho programado, de nuestra capacidad por hora, y luego realizamos una sumatoria para sacar el **balance consolidado de energía por planta y unidad**.

Como es usual, primero diseñamos la query en SQL, dentro del parentesis grande la **subquery** une la tabla del despacho del regulador, con el archivo de capacidades, usando el **código, el año, el mes, el día y la hora**.

Posteriormente, convertimos la **capacidad** que viene como texto con **cast** y restamos el valor de la **capacidad reportada por el regulador**, de la **capacidad real de mi archivo**, y eso lo llamamos **balance_disponible_horario**.

En la query externa (fuera del parentesis), sacamos la **Sumatoria con SUM** y ese es el valor **consolidado del día de nuestra planta en Kilowatts Hora**.

---
##Ejercicio

1. En la siguiente celda toma la query y completa luego del **ON** las condiciones para unir **a** con **b**, ej: **a.atributo >= b.atributo**, recuerda encadenarlos con un **AND**
2. **Prueba tu query** y valida que te esta trayendo los datos de balances para las plantas y las columnas **anio**, **mes**, **día**, **código** y **consolidado_planta**.
3. En la celda que le sigue completa el código para incorporar la **query que acabas de probar** y generar el dataframe **dfBalanceConsolidado**.
4. Crea la tabla temporal **BalanceConsolidado** del dataframe **dfBalanceConsolidado**.
---

In [0]:
%sql
SELECT c.anio, c.mes, c.dia, c.codigo, SUM(c.balance_disponible_horario) AS consolidado_planta
FROM
(
  SELECT a.anio, a.mes, a .dia, a.hora, a.codigo, cast(a.capacidad AS DECIMAL) - cast(b.capacidad as DECIMAL) as balance_disponible_horario FROM ArchivoCapacidadLimpiado a JOIN DespachosAcmeTransformado b
  ON
  --Tu código aquí--
) c 
GROUP BY c.anio, c.mes, c.dia, c.codigo 



---
## *Desafío*

Realiza la misma query usando solo **Pyspark**, puedes hacerlo en una sola etapa o en 2 separadas, GO!.

---

Si la query anterior funciona, la guardamos con Spark, en el dataframe **dfBalanceConsolidado** y creamos la tabla **BalanceConsolidado**

In [0]:
#Tu código va aquí
#dfBalanceConsolidado = <Aqui usa spark para crear el dataframe>
#dfBalanceConsolidado.<Aqui a partir del dataframe crea la tabla.>


Ahora con el precio de la bolsa que está en la tabla **Precios Bolsa** vamos a **multiplicar** de la capacidad en el **Balance Consolidado** que acabamos de calcular y eso nos dará el valor de la energía en miles de millones de pesos.

---
## Ejercicio

En la siguiente query, a veces salen valores nulos, porque?.
Que cambios usarías para evitar que a veces salgan nulos?.
Implementa los cambios!.

---

In [0]:
%sql
SELECT c.anio, c.mes, c.dia, c.codigo, c.consolidado_planta, (c.consolidado_planta * b.valor)/1000 as Compromisos_MCOP
FROM BalanceConsolidado c JOIN 
(
SELECT DAY(a.Fecha) as dia,Month(a.Fecha) as mes,Year(a.Fecha) as anio, a.Valor as valor 
FROM PreciosBolsa a 
WHERE a.CodigoVariable ="PPBOGReal" AND Version="TXR"
) b 
ON c.dia = b.dia AND
c.mes = b.mes AND
c.anio = b.anio


Aqui guardamos lo que hicimos en SQL en el dataframe **BalancesComprasVentasEnergía**

---
##Ejercicio
1. Usa la query anterior y completa la creación del dataframe **dfBalancesComprasVentasEnergia**
2. Usa el dataframe como es usual y crea la tabla temporal **BalancesComprasVentasEnergia**

---

In [0]:
#Tu código Aquí
#dfBalancesComprasVentasEnergia=<Aca el código para crear el dataframe de spark>
#dfBalancesComprasVentasEnergia.<Crea la tabla BalancesComprasVentasEnergia>

Finalmente, para crear un reporte que vamos a enviar al regulador, vamos a agregar una columna **Operación**, que consistirá en evaluar, si el **compromiso es negativo**, debemos comprar energía (la planta no genera lo planeado y hay que ir a bolsa), si el **compromiso es positivo**, tenemos un excedente, podemos vender esa energía en bolsa.

In [0]:
%sql
SELECT a.*, CASE WHEN a.Compromisos_MCOP < 0 THEN "Comprar" ELSE "Vender" END as Operacion FROM BalancesComprasVentasEnergia a

Preparamos el Data Frame de Salida **dfReporteCompraVentaEnergiaAcme** para poderlo enviar en nuestra etapa **L** al destino

##Ejercicio
1. Toma la query anterior y crea el dataframe que vamos a convertir en el reporte del balance **dfReporteCompraVentaEnergiaAcme**
2. Ejecuta la celda con display, y mira los resultados, tienen sentido?, que ha pasado?.

In [0]:
#dfReporteCompraVentaEnergiaAcme=<Usa la query anterior para crear el dataframe aqui>

In [0]:
display(dfReporteCompraVentaEnergiaAcme)

##(*L*) Transferencia de Archivo de Resultados a Destino

Ahora tomamos el resultado de las compras y ventas de energía y lo transferimos al destino para que lo cargue el usuario final.

Vamos a simular la carga del archivo a un servicio de transferencia de archivos llamado https://fastupload.io 

Vamos a **obtener el CSV**, para ello vamos a usar python para **validar si nuestro archivo es un CSV y si es, lo copiamos** a una ruta que podamos acceder.

In [0]:
local_csv_path = "dbfs:/reporteSalidaRegulador/"
dfReporteCompraVentaEnergiaAcme.coalesce(1).write.option("header", "true").mode("overwrite").csv(local_csv_path)
filepath=""
for archivo in dbutils.fs.ls("/reporteSalidaRegulador"):
 if '.csv' in archivo.name:
    filepath=archivo.path
print(filepath)

dbutils.fs.cp(filepath, f"file:/tmp/dfReporteCompraVentaEnergiaAcme.csv")

## *(L)* Autenticación en el sitio remoto del ente regulador

Para poder **cargar archivos**, hay que **iniciar sesión** en la plataforma, para ello se utiliza un API, https://fastupload.io/api#authorize , en el api, la plataforma iniciará sesión y obtendrá un token (secuencia de texto y números) que nos permitirá subir el archivo.


---
##Ejercicio

1. En el siguiente código, utiliza el objeto **json_response** y carga en el, el texto del **response** obtenido, en formato **json** (puedes buscar en internet o consultar).

---


In [0]:
file_upload_access_token = ""
file_upload_account_id = ""
params = {'key1': api_key_1, 'key2': api_key_2}
response=requests.get("https://fastupload.io/api/v2/authorize", params)
#json_response = <Acá va tu código que toma el response, lo convierte en JSON y lo sube a json_response>
try:
  file_upload_access_token = json_response["data"]["access_token"]
  file_upload_account_id = json_response["data"]["account_id"]
except:
  print("Error autenticando y autorizando en el servicio remoto de carga de archivos.")


Así se ve un token de un API 

In [0]:
print(file_upload_access_token)

## *(L)* Ahora vamos a cargar el Archivo finalmente con el token

Para ello usamos otra API https://fastupload.io/api#file-upload, la cual nos pide algunos parámetros, como el nombre del archivo, el token, la cuenta y el folder, al final, al imprimir la respuesta, veremos que el arhivo fue cargado "File Uploaded".

In [0]:
upload_folder_id = ""
json_response=""
with open("/tmp/dfReporteCompraVentaEnergiaAcme.csv", "rb") as archivo:
  files = {"upload_file": (archivo.name, archivo)}  # Create a dictionary for the file upload

  data = {
    "access_token": file_upload_access_token,
    "account_id": file_upload_account_id,
    "folder_id": upload_folder_id  # Include folder_id if provided
  }

  response = requests.post("https://fastupload.io/api/v2/file/upload", files=files, data=data)
  
print(response.text)
